## Tutorial: AGN Photometry with Metropolis Sampling

Note: this tutorial follows [AGN Photometry on a Grid](agn_photometry_grid.ipynb). Along with [AGN Photometry with Gibbs Sampling](agn_photometry_gibbs.ipynb), it should be done before [MCMC Diagnostics](mcmc_diagnostics.ipynb).

Having laboriously done Baysian inference on a grid to fit an AGN source to X-ray data, we will now turn to solving the same problem using the Metropolis sampling method of MCMC.

In [ ]:
exec(open('tbc.py').read()) # define TBC and TBC_above
import astropy.io.fits as pyfits
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO   # StringIO behaves like a file object
import scipy.stats as st
%matplotlib inline
import corner
import incredible as cr

Once again, we will read in the X-ray image data, and extract a small image around an AGN that we wish to study.

In [ ]:
# get the Image object defined in the "X-ray image introduction" notebook, for convenience
from xray_image import Image

In [ ]:
TBC() # datadir = '../ignore/' # or whatever - path to where you put the downloaded files

imagefile = datadir + 'P0098010101M2U009IMAGE_3000.FTZ'
expmapfile = datadir + 'P0098010101M2U009EXPMAP3000.FTZ'

imfits = pyfits.open(imagefile)
exfits = pyfits.open(expmapfile)

im = imfits[0].data
ex = exfits[0].data

orig = Image(im, ex)

In [ ]:
x0 = 417
y0 = 209
stampwid = 25
stamp = orig.cutout(x0-stampwid, x0+stampwid, y0-stampwid, y0+stampwid)

plt.rcParams['figure.figsize'] = (10.0, 10.0)
stamp.display(log_image=False)

## Model implementation

This time we're going to do a fit using the Metropolis algorithm. This doesn't require any of the restrictive assumptions that were needed to demonstrate conjugate Gibbs sampling, so let's go straight to fitting the whole model from the [Grid](agn_photometry_grid.ipynb) notebook on this subimage.

In [ ]:
params = {'x0':x0, 'y0':y0, 'lnF0':-5.0, 'b':1e-6, 'sigma':1.25}

As before, we will assume uniform priors on all parameters as defined below, for simplicity. Since we're not confined to a grid, also include reasonable, wide boundaries (beyond the physical requirements $b>=0$ and $\sigma>0$). You can define a `hyperparams` dictionary to hold the endpoints, or simply include them directly in the `log_prior` function, below.

In [ ]:
def log_prior(x0, y0, lnF0, b, sigma):
    TBC()
    
TBC_above()

The `log_likelihood` function can very likely be recycled from your `agn_photometry_grid` notebook without changes.

In [ ]:
def log_likelihood(data, x0, y0, lnF0, b, sigma):
    TBC()
    
TBC_above()

And here is the free log-posterior function again:

In [ ]:
def log_posterior(data, **params):
    lnp = log_prior(**params)
    if np.isfinite(lnp):
        lnp += log_likelihood(data, **params)
    return lnp

As always, let's check that they return finite values without obvious bugs.

In [ ]:
print( log_prior(**params) )
print( log_likelihood(stamp, **params) )
print( log_posterior(stamp, **params) )

## Sampler implementation

Next, we need a proposal distribution. I'll use a multivariate Gaussian centered on the current position. This is translationally invariant, so later on we can use the simple Metropolis acceptance rule instead of the slightly more complex Metropolis-Hastings rule. A Gaussian isn't necessarily the best choice in general, since the most likely proposals are very small steps, but it will do for the moment.

To further keep it simple, let's make the proposal independent in each parameter (a diagonal covariance matrix for the 5-dimensional Gaussian). Similarly to the grid method, you'll want to guess the appropriate order of magnitude for steps in each parameter, which is the same order as the width of the posterior, and you may need to return to this point to adjust them after seeing the performance.

Since we're assuming a diagonal covariance, let's go ahead and just represent the proposal distribution as 5 univariate Gaussians, as below.

In [ ]:
TBC()
# proposal_distribution = {'x0':st.norm(...  ,
#                          'y0':st.norm(... ,
#                          'lnF0':st.norm(... ,
#                          'b':st.norm(... ,
#                          'sigma':st.norm(... )}

Next, define a function that returns a proposed point in parameter space, given the current location and the above dictionary of proposal distributions.

_Technical note:_ Remember that, in Python, `b = a` does not make a copy of `a` if `a` is a dictionary (or a `numpy` array for that matter). Both `b` and `a` would point to the same data in memory. The safest/quickest way to get a new dictionary with the same structure as `a` whose values can then be overwritten is `b = a.copy()`;

In [ ]:
def propose(current_params, dists):
    """
    current_params: dictionary holding current position in parameter space
    dists: dictionary of proposal distributions
    
    Return value: a new dictionary holding the proposed destination in parameter space
    """
    TBC()
    
TBC_above()

See if it works:

In [ ]:
propose(params, proposal_distribution)

Finally, the sampler itself. Write a function that takes the current parameter values and log-posterior value as input (along with the data and proposal distributions), and returns the next set of parameters values and corresponding log-posterior. These can be identical to the inputs, if the proposal is rejected.

In [ ]:
def step(data, current_params, current_lnP, proposal_dists):
    """
    data: Image object
    current_params: dictionary of parameter values
    current_lnP: log-posterior density corresponding to current_params
    proposal_dists: dictionary of proposal distributions
    
    Return value: a tuple holding the next parameter dictionary and corresponding log-posterior density
    """
    TBC()
    # trial_params = ...
    # trial_lnP = ...
    # if [accept/reject condition]:
    #    return (trial_params, trial_lnP)
    # else:
    #    return (current_params, current_lnP)
    
TBC_above()

And, again, make sure it works without crashing:

In [ ]:
step(stamp, params, log_posterior(stamp, **params), proposal_distribution)

## Results

Assuming everything above looks ok, let's run a test chain.

In [ ]:
%%time
nsamples = 10000

current_lnP = log_posterior(stamp, **params)
samples = np.zeros((nsamples, len(params)))
for i in range(samples.shape[0]):
    params, current_lnP = step(stamp, params, current_lnP, proposal_distribution)
    samples[i,:] = [params['x0'], params['y0'], params['lnF0'], params['b'], params['sigma']]

Let's do the most basic (yet still extremely important) visual check to see how our sampler performed, looking at traces of the Markov chain for each parameter. (It's ok if you haven't read the notes on [MCMC Diagnostics](../notes/mcmc_diagnostics.ipynb) yet; we will go more in-depth later.) These trace plots show the value of each parameter as a function of iteration.

In [ ]:
param_labels = [r'$x_0$', r'$y_0$', r'$\ln F_0$', r'$b$', r'$\sigma$']
plt.rcParams['figure.figsize'] = (16.0, 12.0)
fig, ax = plt.subplots(len(param_labels), 1);
cr.plot_traces(samples, ax, labels=param_labels)

Exactly what you see here will depend on the width of your proposal distributions. But, hopefully, you can see the sequence finding its way to a part of parameter space that it likes, and then continuing to jump around there.

We can use `corner` to quickly visualize the posterior. This package shows us all the 1D marginalized posteriors (as histograms) and every pair of 2D marginalized posteriors (as a contour plot) in a triangular grid. Note that we should really remove the burn-in phase before doing this, but this is just for a quick look.

In [ ]:
corner.corner(samples, labels=param_labels);

If you've already worked through the Gibbs sampling notebook, how do these traces compare qualitatively? What role do you think the proposal distribution width plays? Feel free to tweak those widths and re-run (from the same starting point) to see how things change. You can also change the chain length if you think that will help explore the parameter space better.

We weren't overly concerned with the starting point for the test chain above. But, for later notebooks, we'll want to see how multiple, independent chains with different starting points behave when using this method. The cell below will take care of running 4 chains, started at random positions broadly in the vicinity of the input values.

In [ ]:
%%time
chains = [np.zeros((nsamples, len(params))) for j in range(4)]

for samples in chains:
    params = {'x0':st.uniform.rvs()*20.0 + 410.0,
              'y0':st.uniform.rvs()*20.0 + 200.0,
              'lnF0':st.uniform.rvs()*3.0 -8.0,
              'b':st.uniform.rvs()*5e-6 + 3e-6,
              'sigma':st.uniform.rvs()*4.9 + 0.1}
    current_lnP = log_posterior(stamp, **params)
    for i in range(samples.shape[0]):
        params, current_lnP = step(stamp, params, current_lnP, proposal_distribution)
        samples[i,:] = [params['x0'], params['y0'], params['lnF0'], params['b'], params['sigma']]

Now we can look at a more colorful version of the trace plots, showing all of the chains simultaneously:

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 12.0)
fig, ax = plt.subplots(len(param_labels), 1);
cr.plot_traces(chains, ax, labels=param_labels, Line2D_kwargs={'markersize':1.0})

Save them for later, and we're done!

In [ ]:
TBC() # change path below, if desired
#for i,samples in enumerate(chains):
#    np.savetxt('../ignore/agn_metro_chain_'+str(i)+'.txt', samples, header='x0 y0 lnF0 b sigma')

There you have it - at the cost of a little random walking, you've now fit a model with enough free parameters to make a grid-based solution uncomfortably slow.